In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4180/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4180/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_4_6_7,0.986219,0.758475,0.895286,0.978710,0.009166,0.143380,0.089189,0.014296,0.067042,0.095741,1.026727,0.097249,203.384431,414.299996,"Hidden Size=[24], regularizer=0.02, learning_r..."
1,model_4_6_3,0.985106,0.758271,0.894368,0.977123,0.009907,0.143501,0.089971,0.015362,0.069680,0.099532,1.028885,0.101099,203.229123,414.144689,"Hidden Size=[24], regularizer=0.02, learning_r..."
2,model_4_6_0,0.984229,0.758082,0.893675,0.975918,0.010490,0.143613,0.090561,0.016171,0.071696,0.102420,1.030586,0.104033,203.114682,414.030247,"Hidden Size=[24], regularizer=0.02, learning_r..."
3,model_4_5_7,0.983253,0.757960,0.892882,0.984646,0.011139,0.143685,0.091237,0.015353,0.073909,0.105541,1.032478,0.107203,202.994635,413.910200,"Hidden Size=[24], regularizer=0.02, learning_r..."
4,model_4_5_3,0.981836,0.757809,0.891741,0.983309,0.012081,0.143775,0.092209,0.016689,0.077041,0.109916,1.035227,0.111647,202.832166,413.747731,"Hidden Size=[24], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,model_1_0_0,0.395066,0.377644,0.268834,0.273341,0.402367,0.369457,0.622764,0.772882,0.389185,0.634324,0.176260,0.644314,35.820782,72.785365,"Hidden Size=[4], regularizer=0.02, learning_ra..."
650,model_2_0_5,0.523009,0.343142,0.469977,0.427091,0.317266,0.389939,0.451442,0.251193,0.443803,0.563264,0.015245,0.572135,68.296028,140.050808,"Hidden Size=[8], regularizer=0.02, learning_ra..."
651,model_1_1_5,0.506620,0.334413,0.479100,0.115500,0.328168,0.395121,0.443672,0.531098,0.454978,0.572859,0.328163,0.581881,36.228462,73.193046,"Hidden Size=[4], regularizer=0.02, learning_ra..."
652,model_2_0_0,0.288981,0.136310,0.414231,0.166613,0.472929,0.512724,0.498923,0.365400,0.523303,0.687698,-0.467911,0.698529,67.497622,139.252401,"Hidden Size=[8], regularizer=0.02, learning_ra..."
